In [1]:
from glob import glob
import pandas as pd
import numpy as np
from Bio import PDB
import os
from rdkit.Chem.rdmolfiles import MolFromSmiles, MolToSmiles
from Bio.PDB import Select, PDBIO
from Bio.PDB.PDBParser import PDBParser
from biopandas.mol2 import PandasMol2

### 1. Load complex list

In [2]:
pdb_path = os.path.abspath("../datasets/examples/scPDB/")
info_path = os.path.abspath("../datasets/examples/")

In [3]:
complex_list = os.listdir(pdb_path)

### 2. Run Chimera

In [4]:
%%bash -s $pdb_path
path=$1

for file in $path/*/'protein.mol2';do
    output_path=${file%.mol2}".pdb"
    echo -e "open $file \n write format pdb 0 $output_path \n stop" | chimera --nogui
done > chimera.log

### 3. Remove HEATM

In [5]:
def remove_HEATM_PDBbind(input_list, path):

    class NonHetSelect(Select):
        def accept_residue(self, residue):
            return 1 if residue.id[0] == " " else 0
    
    for pdb in input_list:

        src_file = f"{pdb_path}/{pdb}/protein.pdb"
        des_file = f"{pdb_path}/{pdb}/remove_HEATM_protein.pdb"
        
        pdb = PDBParser().get_structure(pdb, src_file)
        io = PDBIO()
        io.set_structure(pdb)
        io.save(des_file, NonHetSelect()) 

In [6]:
remove_HEATM_PDBbind(complex_list, pdb_path)

/home/ssm/anaconda3/envs/cma/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 17538.
  PDBConstructionWarning,
/home/ssm/anaconda3/envs/cma/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 17655.
  PDBConstructionWarning,
/home/ssm/anaconda3/envs/cma/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 17772.
  PDBConstructionWarning,
/home/ssm/anaconda3/envs/cma/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 17889.
  PDBConstructionWarning,
/home/ssm/anaconda3/envs/cma/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 17962.
  PDBConstructionWarning,
/home/ssm/anaconda3/envs/cma/lib/python3.7/site-packages/Bio

### 3. Load Binding sites info

In [7]:
from scipy.spatial import distance_matrix
from multiprocessing import Process, Queue, Pool

In [8]:
pdb_parser = PDB.PDBParser(QUIET=True)

In [9]:
amino_acids_short = {"ALA":"A", "ARG":"R", "ASN":"N", "ASP":"D", "CYS":"C", "GLU":"E", "GLN":"Q", "GLY":"G", "HIS":"H", "ILE":"I", "LEU":"L", "LYS":"K", "MET":"M", "PHE":"F", "PRO":"P", "SER":"S", "THR":"T", "TRP":"W", "TYR":"Y", "VAL":"V", "SEC":"U", "PYL":"O"}

In [10]:
def parallelize_dataframe(df, func, num_partitions=10):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_partitions)
    results = pool.map(func, df_split)
    pool.close()
    pool.join()
    return results

In [11]:
def get_protein(scpdb):
    try:
        """ Load protein info """
        structure = pdb_parser.get_structure(scpdb, f"{pdb_path}/{scpdb}/remove_HEATM_protein.pdb")
        chain_name_list, pdb_sequence_list, seq_lengths_list, protein_atom_coords, protein_residue_list, reindex = list(), list(), list(), list(), list(), 0

        for chain_name in list(structure[0].child_dict.keys()):
            chain = structure[0][chain_name]

            residue_number, pdb_sequence = list(), ""
            for residue in chain.get_residues():
                if residue.resname in amino_acids_short.keys():
                    residue_number.append(reindex)
                    pdb_sequence += amino_acids_short[residue.resname]

                    for atom in residue:
                        protein_atom_coords.append(atom.get_coord())
                        protein_residue_list.append(reindex)
                    reindex += 1

            if len(pdb_sequence) != 0:
                chain_name_list.append(chain_name)
                pdb_sequence_list.append(pdb_sequence)
                seq_lengths_list.append(len(pdb_sequence)) 
                #residue_number_list.append(",".join(map(str, residue_number)))

        """ Load pocket info """
        bi_df = PandasMol2().read_mol2(f"{pdb_path}/{scpdb}/site.mol2").df
        bi_df = bi_df.drop_duplicates("subst_name")
        bi_df = bi_df.loc[bi_df.subst_name.map(lambda a: a[:3] in amino_acids_short.keys())]
        bi_xyz_coordi = bi_df.iloc[:, [2,3,4]].values
        bi_x, bi_y, bi_z = bi_xyz_coordi[:, 0], bi_xyz_coordi[:, 1], bi_xyz_coordi[:, 2] 

        protein_atom_coords, binding_index = np.array(protein_atom_coords), list()

        for i, j, k in zip(bi_x, bi_y, bi_z):

            tmp_coordi = np.array([i, j, k], dtype = np.float32)
            ind = np.where((protein_atom_coords == tmp_coordi).all(axis = 1))[0][0]
            binding_index.append(protein_residue_list[ind])

        binding_index = list(map(str, binding_index))

        total_seq_lengths = np.sum(np.array(seq_lengths_list))
        seq_lengths_list = list(map(str, seq_lengths_list))        
        
        return ",".join(chain_name_list), ",".join(pdb_sequence_list), total_seq_lengths, ",".join(seq_lengths_list), ",".join(binding_index)
    
    except Exception as e:
        print(scpdb, e)
        return None

In [12]:
def get_raw_protein_info_bulk(df):
    return df.scPDB.map(get_protein)

In [13]:
pdb = [scpdb[:-2] for scpdb in complex_list]

In [14]:
data_df = pd.DataFrame({"scPDB":complex_list, "PDB":pdb})
data_df

,scPDB,PDB
0,3apy_3,3apy
1,1fdu_4,1fdu
2,1n2g_2,1n2g
3,2e9n_1,2e9n
4,4ear_3,4ear
5,1fdu_5,1fdu
6,1n22_2,1n22
7,1d0v_1,1d0v
8,2e82_4,2e82
9,2x7s_2,2x7s


In [15]:
info_results = parallelize_dataframe(data_df, get_raw_protein_info_bulk, num_partitions = 5)

In [16]:
info_results = pd.concat(info_results)

In [17]:
data_df["Chain"] = info_results.map(lambda a: a[0] if a is not None else None)

In [18]:
data_df["Sequence"] = info_results.map(lambda a: a[1] if a is not None else None)

In [19]:
data_df["Total_seq_lengths"] = info_results.map(lambda a: a[2] if a is not None else None)

In [20]:
data_df["Chain_seq_lengths"] = info_results.map(lambda a: a[3] if a is not None else None)

In [21]:
data_df["BS"] = info_results.map(lambda a: a[4] if a is not None else None)

In [22]:
data_df = data_df.loc[data_df.Sequence.isna()==False].reset_index(drop=True)
data_df = data_df.loc[data_df.Chain != " "].reset_index(drop=True)

In [23]:
data_df

,scPDB,PDB,Chain,Sequence,Total_seq_lengths,Chain_seq_lengths,BS
0,3apy_3,3apy,C,MKIRDLLKARRGPLFSFEFFPPKDPEGEEALFRTLEELKAFRPAFV...,291,291,"17,47,48,49,50,51,76,77,78,103,104,105,106,107..."
1,1fdu_4,1fdu,"A,B,C,D",ARTVVLITGCSSGIGLHLAVRLASDPSQSFKVYATLRDLKTQGRLW...,1127,"281,280,285,281","986,987,988,989,990,992,994,997,1000,1001,1004..."
2,1n2g_2,1n2g,"A,B",IPAFHPGELNVYSAPGDVADVSRALRLTGRRVMLVPTMGALHEGHL...,573,"289,284","324,325,326,327,328,329,331,332,333,334,336,33..."
3,2e9n_1,2e9n,A,AVPFVEDWDLVQTLGEGAYGEVQLAVNRVTEEAVAVKIVDMKRAVD...,269,269,"13,14,18,21,23,33,34,36,50,53,54,57,66,80,82,8..."
4,4ear_3,4ear,"B,C",ENGYTYEDYKNTAEYLLSHTKHRPQVAIICGSGLGGLTDKLTQAQI...,570,"286,284","157,315,343,344,346,368,370,397,398,399,400,40..."
5,1fdu_5,1fdu,"A,B,C,D",ARTVVLITGCSSGIGLHLAVRLASDPSQSFKVYATLRDLKTQGRLW...,1127,"281,280,285,281","7,8,9,10,11,12,13,14,15,34,35,36,37,40,63,64,6..."
6,1n22_2,1n22,"A,B",IRRSGNYQPALWDSNYIQSLNTPYTEERHLDRKAELIVQVRILLKE...,1056,"534,522","776,782,785,805,806,807,809,810,813,817,888,91..."
7,1d0v_1,1d0v,A,LHALLRDIPAPDAEAMARTQQHIDGLLKPPGSLGRLETLAVQLAGM...,346,346,"69,73,74,75,76,80,81,84,88,170,171,172,173,174..."
8,2e82_4,2e82,"A,B,C,D",MRVVVIGAGVIGLSTALCIHERYHSVLQPLDIKVYADRFTPLTTTD...,1360,"340,340,340,340","1068,1070,1072,1073,1074,1075,1115,1221,1234,1..."
9,2x7s_2,2x7s,A,HWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSV...,256,256,"54,56,58,61,62,63,64,65,66,68,86,87,89,116,118..."


### 4. Remove complex over than 1,500 protein seq length

In [24]:
lengths = data_df.Total_seq_lengths.values

In [25]:
data_df = data_df[lengths <= 1500].reset_index(drop=True)
data_df

,scPDB,PDB,Chain,Sequence,Total_seq_lengths,Chain_seq_lengths,BS
0,3apy_3,3apy,C,MKIRDLLKARRGPLFSFEFFPPKDPEGEEALFRTLEELKAFRPAFV...,291,291,"17,47,48,49,50,51,76,77,78,103,104,105,106,107..."
1,1fdu_4,1fdu,"A,B,C,D",ARTVVLITGCSSGIGLHLAVRLASDPSQSFKVYATLRDLKTQGRLW...,1127,"281,280,285,281","986,987,988,989,990,992,994,997,1000,1001,1004..."
2,1n2g_2,1n2g,"A,B",IPAFHPGELNVYSAPGDVADVSRALRLTGRRVMLVPTMGALHEGHL...,573,"289,284","324,325,326,327,328,329,331,332,333,334,336,33..."
3,2e9n_1,2e9n,A,AVPFVEDWDLVQTLGEGAYGEVQLAVNRVTEEAVAVKIVDMKRAVD...,269,269,"13,14,18,21,23,33,34,36,50,53,54,57,66,80,82,8..."
4,4ear_3,4ear,"B,C",ENGYTYEDYKNTAEYLLSHTKHRPQVAIICGSGLGGLTDKLTQAQI...,570,"286,284","157,315,343,344,346,368,370,397,398,399,400,40..."
5,1fdu_5,1fdu,"A,B,C,D",ARTVVLITGCSSGIGLHLAVRLASDPSQSFKVYATLRDLKTQGRLW...,1127,"281,280,285,281","7,8,9,10,11,12,13,14,15,34,35,36,37,40,63,64,6..."
6,1n22_2,1n22,"A,B",IRRSGNYQPALWDSNYIQSLNTPYTEERHLDRKAELIVQVRILLKE...,1056,"534,522","776,782,785,805,806,807,809,810,813,817,888,91..."
7,1d0v_1,1d0v,A,LHALLRDIPAPDAEAMARTQQHIDGLLKPPGSLGRLETLAVQLAGM...,346,346,"69,73,74,75,76,80,81,84,88,170,171,172,173,174..."
8,2e82_4,2e82,"A,B,C,D",MRVVVIGAGVIGLSTALCIHERYHSVLQPLDIKVYADRFTPLTTTD...,1360,"340,340,340,340","1068,1070,1072,1073,1074,1075,1115,1221,1234,1..."
9,2x7s_2,2x7s,A,HWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSV...,256,256,"54,56,58,61,62,63,64,65,66,68,86,87,89,116,118..."


### 5. Remove complex over than 160 SMILES length

In [26]:
from rdkit.Chem.rdmolfiles import MolFromSmiles, MolToSmiles
def read_file(file):
    return file.readlines()

def add_ligand(scpdb):

    mol = f"{pdb_path}/{scpdb}/ligand.mol2"

    #command = f"obabel -imol2 {mol} -osmi -O tmp.smi"
    command = f"obabel -imol2 {mol} -osmi -xC | obabel -ismi -osmi -xk -O tmp.smi"
    os.system(command)

    smiles = read_file(open("tmp.smi"))[0].split('\t')[0].strip()
    
    try:
        smiles = MolToSmiles(MolFromSmiles(smiles),isomericSmiles = False, kekuleSmiles = True)
        return smiles
    
    except Exception as e:
        print(scpdb, e)
        return None 

In [27]:
SMILES = data_df.scPDB.map(add_ligand)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted


In [28]:
data_df["SMILES"] = SMILES

In [29]:
data_df = data_df.loc[data_df.SMILES.isna()==False].reset_index(drop=True)
data_df

,scPDB,PDB,Chain,Sequence,Total_seq_lengths,Chain_seq_lengths,BS,SMILES
0,3apy_3,3apy,C,MKIRDLLKARRGPLFSFEFFPPKDPEGEEALFRTLEELKAFRPAFV...,291,291,"17,47,48,49,50,51,76,77,78,103,104,105,106,107...",CC1=CC2=C(C=C1C)N(CC(O)C(O)C(O)CO[PH](O)(O)O[P...
1,1fdu_4,1fdu,"A,B,C,D",ARTVVLITGCSSGIGLHLAVRLASDPSQSFKVYATLRDLKTQGRLW...,1127,"281,280,285,281","986,987,988,989,990,992,994,997,1000,1001,1004...",CC12CCC3C4=CC=C(O)C=C4CCC3C1CCC2O
2,1n2g_2,1n2g,"A,B",IPAFHPGELNVYSAPGDVADVSRALRLTGRRVMLVPTMGALHEGHL...,573,"289,284","324,325,326,327,328,329,331,332,333,334,336,33...",NC1=C2N=CN(C3OC(CO[PH](O)(O)C[PH](O)(O)O[PH](O...
3,2e9n_1,2e9n,A,AVPFVEDWDLVQTLGEGAYGEVQLAVNRVTEEAVAVKIVDMKRAVD...,269,269,"13,14,18,21,23,33,34,36,50,53,54,57,66,80,82,8...",O=C(NC1CCC(O)CC1)C1=CC2=CC3=C(C4=CC=C(C5=CC=C(...
4,4ear_3,4ear,"B,C",ENGYTYEDYKNTAEYLLSHTKHRPQVAIICGSGLGGLTDKLTQAQI...,570,"286,284","157,315,343,344,346,368,370,397,398,399,400,40...",NC1=NC2=C(NC=C2C[NH+]2CC(O)C(CO)C2)C(=O)N1
5,1fdu_5,1fdu,"A,B,C,D",ARTVVLITGCSSGIGLHLAVRLASDPSQSFKVYATLRDLKTQGRLW...,1127,"281,280,285,281","7,8,9,10,11,12,13,14,15,34,35,36,37,40,63,64,6...",NC(=O)C1=CC=C[N+](C2OC(CO[PH](O)(O)O[PH](O)(O)...
6,1n22_2,1n22,"A,B",IRRSGNYQPALWDSNYIQSLNTPYTEERHLDRKAELIVQVRILLKE...,1056,"534,522","776,782,785,805,806,807,809,810,813,817,888,91...",CC1=CCC([NH+](C)C)CC1
7,1d0v_1,1d0v,A,LHALLRDIPAPDAEAMARTQQHIDGLLKPPGSLGRLETLAVQLAGM...,346,346,"69,73,74,75,76,80,81,84,88,170,171,172,173,174...",CC1CC2NCN(C3OC(CO[PH](O)(O)O)C(O)C3O)C2CC1C
8,2e82_4,2e82,"A,B,C,D",MRVVVIGAGVIGLSTALCIHERYHSVLQPLDIKVYADRFTPLTTTD...,1360,"340,340,340,340","1068,1070,1072,1073,1074,1075,1115,1221,1234,1...",N=C(CC1=CC=C(O)C(O)=C1)C(=O)O
9,2x7s_2,2x7s,A,HWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSV...,256,256,"54,56,58,61,62,63,64,65,66,68,86,87,89,116,118...",COC1=C(O)C=C2CCC3C(CCC4(C)C(OS(N)(=O)=O)CCC34)...


In [30]:
def get_SMILES_length(df):
    index = [True if len(smi) <= 160 else False for smi in df.SMILES.values]
    return index

In [31]:
smiles_index = get_SMILES_length(data_df)

In [32]:
data_df = data_df.loc[smiles_index].reset_index(drop=True)

In [33]:
data_df

,scPDB,PDB,Chain,Sequence,Total_seq_lengths,Chain_seq_lengths,BS,SMILES
0,3apy_3,3apy,C,MKIRDLLKARRGPLFSFEFFPPKDPEGEEALFRTLEELKAFRPAFV...,291,291,"17,47,48,49,50,51,76,77,78,103,104,105,106,107...",CC1=CC2=C(C=C1C)N(CC(O)C(O)C(O)CO[PH](O)(O)O[P...
1,1fdu_4,1fdu,"A,B,C,D",ARTVVLITGCSSGIGLHLAVRLASDPSQSFKVYATLRDLKTQGRLW...,1127,"281,280,285,281","986,987,988,989,990,992,994,997,1000,1001,1004...",CC12CCC3C4=CC=C(O)C=C4CCC3C1CCC2O
2,1n2g_2,1n2g,"A,B",IPAFHPGELNVYSAPGDVADVSRALRLTGRRVMLVPTMGALHEGHL...,573,"289,284","324,325,326,327,328,329,331,332,333,334,336,33...",NC1=C2N=CN(C3OC(CO[PH](O)(O)C[PH](O)(O)O[PH](O...
3,2e9n_1,2e9n,A,AVPFVEDWDLVQTLGEGAYGEVQLAVNRVTEEAVAVKIVDMKRAVD...,269,269,"13,14,18,21,23,33,34,36,50,53,54,57,66,80,82,8...",O=C(NC1CCC(O)CC1)C1=CC2=CC3=C(C4=CC=C(C5=CC=C(...
4,4ear_3,4ear,"B,C",ENGYTYEDYKNTAEYLLSHTKHRPQVAIICGSGLGGLTDKLTQAQI...,570,"286,284","157,315,343,344,346,368,370,397,398,399,400,40...",NC1=NC2=C(NC=C2C[NH+]2CC(O)C(CO)C2)C(=O)N1
5,1fdu_5,1fdu,"A,B,C,D",ARTVVLITGCSSGIGLHLAVRLASDPSQSFKVYATLRDLKTQGRLW...,1127,"281,280,285,281","7,8,9,10,11,12,13,14,15,34,35,36,37,40,63,64,6...",NC(=O)C1=CC=C[N+](C2OC(CO[PH](O)(O)O[PH](O)(O)...
6,1n22_2,1n22,"A,B",IRRSGNYQPALWDSNYIQSLNTPYTEERHLDRKAELIVQVRILLKE...,1056,"534,522","776,782,785,805,806,807,809,810,813,817,888,91...",CC1=CCC([NH+](C)C)CC1
7,1d0v_1,1d0v,A,LHALLRDIPAPDAEAMARTQQHIDGLLKPPGSLGRLETLAVQLAGM...,346,346,"69,73,74,75,76,80,81,84,88,170,171,172,173,174...",CC1CC2NCN(C3OC(CO[PH](O)(O)O)C(O)C3O)C2CC1C
8,2e82_4,2e82,"A,B,C,D",MRVVVIGAGVIGLSTALCIHERYHSVLQPLDIKVYADRFTPLTTTD...,1360,"340,340,340,340","1068,1070,1072,1073,1074,1075,1115,1221,1234,1...",N=C(CC1=CC=C(O)C(O)=C1)C(=O)O
9,2x7s_2,2x7s,A,HWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSV...,256,256,"54,56,58,61,62,63,64,65,66,68,86,87,89,116,118...",COC1=C(O)C=C2CCC3C(CCC4(C)C(OS(N)(=O)=O)CCC34)...


### 6. Save data

In [34]:
data_df = data_df.iloc[:, [0, 3, 6]]

In [35]:
data_df.to_csv("../datasets/examples/scPDB_data.tsv", sep = "\t", index = False)